In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import re

In [2]:
def status_check(url):
    import requests
    r=requests.get(url)
    if r.status_code<300:
        print("request was successful")
    elif r.status_code>=400 and r.status_code<500:
        print('request failed because it is non-existant or forbidden')
    else: print('blablabla')
    return r

In [3]:
def get_soup(url):
    r=status_check(url)
    soup = BeautifulSoup(r.content)
    return soup

In [4]:
def get_stars(soup):
    if len(soup.select('h4:contains("Stars")~a'))>1:
        stars= [i.text for i in soup.select('h4:contains("Stars")~a')[:-1]]
    else:
        stars = np.nan
    return stars

In [5]:
def get_writer(soup):
    if len(soup.select('h4:contains("Writer")+a'))>0:
        writer = soup.select('h4:contains("Writer")+a')[0].text
    else:
        writer = np.nan
    return writer

In [6]:
def get_popularity(soup):
    if len(soup.select('div:contains("Popularity")+div>span'))>0:
        popularity = soup.select('div:contains("Popularity")+div>span')[0].text.strip()
    else:
        popularity = np.nan
    return popularity

In [7]:
def get_keywords(soup):
    if len(soup.select('h4:contains("Keywords")~a'))>0:
        plot_keywords = [i.text for i in soup.select('h4:contains("Keywords")~a')]
    else:
        plot_keywords = np.nan
    return plot_keywords

In [8]:
def get_languages(soup):
    if len(soup.select('h4:contains("Language")~a'))>0:
        languages = [i.text for i in soup.select('h4:contains("Language")~a')]
    else:
        languages = np.nan
    return languages

In [9]:
def get_country(soup):
    if len(soup.select('h4:contains("Country")+a'))>0:
        country = soup.select('h4:contains("Country")+a')[0].text
    else:
        country = np.nan
    return country

In [10]:
def get_budget(soup):
    if len(soup.select('div.txt-block:contains("Budget")'))>0:
        test=soup.select('div.txt-block:contains("Budget")')[0].text
        budget=re.findall('Budget:\$([\d\,]*)',test)
    else:
        budget = np.nan
    return budget

In [11]:
def get_cum_gross(soup):
    if len(soup.select('div.txt-block:contains("Worldwide")'))>0:
        test2=soup.select('div.txt-block:contains("Worldwide")')[0].text
        gross=re.findall('Gross: \$([\d\,]*)',test2)
    else:
        gross = np.nan
    return gross

In [12]:
df1=pd.read_csv('movies_imdb.csv')

In [13]:
df1.head()

,movie,year,imdbID,genre,runtime,rating,metascore,director,votes,gross
0,The Dark Knight: Le chevalier noir,(2008),/title/tt0468569/,"\nAction, Crime, Drama",152 min,9.0,84.0,Christopher Nolan,"2,161,024",$534.86M
1,Inception,(2010),/title/tt1375666/,"\nAction, Adventure, Sci-Fi",148 min,8.8,74.0,Christopher Nolan,"1,909,234",$292.58M
2,Le seigneur des anneaux: La communauté de l'an...,(2001),/title/tt0120737/,"\nAdventure, Drama, Fantasy",178 min,8.8,92.0,Peter Jackson,"1,561,138",$315.54M
3,Le seigneur des anneaux: Le retour du roi,(2003),/title/tt0167260/,"\nAdventure, Drama, Fantasy",201 min,8.9,94.0,Peter Jackson,"1,549,252",$377.85M
4,The Dark Knight Rises,(2012),/title/tt1345836/,"\nAction, Thriller",164 min,8.4,78.0,Christopher Nolan,"1,431,808",$448.14M


In [14]:
df1.shape

(6735, 10)

In [15]:
ID_list = df1.imdbID.to_list()

In [16]:
ll=[]
n=1

In [ ]:
for ID in ID_list:
    row = [ID]
    url='https://www.imdb.com'+ID
    print(n)
    soup = get_soup(url)
    row.append(get_stars(soup))
    row.append(get_writer(soup))
    row.append(get_popularity(soup))
    row.append(get_keywords(soup))
    row.append(get_languages(soup))
    row.append(get_country(soup))
    row.append(get_budget(soup))
    row.append(get_cum_gross(soup))
    ll.append(row)
    n+=1
    time.sleep(0.5)

In [140]:
df2=pd.DataFrame(ll)

In [141]:
df2.columns=['imdbID','stars','writer','popularity','plot_keywords','languages','country','budget','cum_worldwide_gross']

In [142]:
df2.to_csv('movies_imdb_more_info.csv', index=False)

In [146]:
df2.head()

,imdbID,stars,writer,popularity,plot_keywords,languages,country,budget,cum_worldwide_gross
0,/title/tt0468569/,"[Christian Bale, Heath Ledger, Aaron Eckhart]",Jonathan Nolan,125\n ( 36),"[ dc comics, joker, psychopath, clown, cri...","[English, Mandarin]",USA,"[185,000,000]","[1,004,934,033]"
1,/title/tt1375666/,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",Christopher Nolan,89\n ( 68),"[ dream, subconscious, ambiguous ending, th...","[English, Japanese, French]",USA,"[160,000,000]","[829,895,144]"
2,/title/tt0120737/,"[Elijah Wood, Ian McKellen, Orlando Bloom]",J.R.R. Tolkien,113\n ( 14),"[ ring, quest, hobbit, elf, orc]","[English, Sindarin]",New Zealand,"[93,000,000]","[872,491,916]"
3,/title/tt0167260/,"[Elijah Wood, Viggo Mortensen, Ian McKellen]",J.R.R. Tolkien,173\n ( 63),"[ orc, epic, battle, journey, hobbit]","[English, Quenya, Old English, Sindarin]",New Zealand,"[94,000,000]","[1,120,424,614]"
4,/title/tt1345836/,"[Christian Bale, Tom Hardy, Anne Hathaway]",Jonathan Nolan,175\n ( 171),"[ dc comics, batman character, terrorist plo...","[English, Arabic]",UK,"[250,000,000]","[1,081,041,287]"
